# Evaluation of models on test sets

## Importing modules

In [ ]:
from utils.miscellaneous import read_config
from sklearn.metrics import confusion_matrix

import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import utils.train_val_test_dataset_import as tvt
import utils.evaluation_matrix as em
import cv2 


##  1 Parse configuration file + initializations

In [ ]:
# read config files
cfg = read_config('./config.yaml')

# constants
num_classes = cfg['num_classes']
image_height = cfg['image_height']
image_width = cfg['image_width']
batch_size = cfg['batch_size']['tra']

labels = cfg['labels']

# paths
path_test = cfg['Path']['path_test']

save_misclassified_images_path = cfg['Path']['save_misclassified_images_path']
model_path = cfg['Path']['model_path']


## 2 Load test dataset

In [ ]:
ds_test = tvt.import_dataset_test(
  path_test, image_height, image_width, batch_size)

# autotune
AUTOTUNE = tf.data.AUTOTUNE
ds_test = ds_test.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Visualizing the test DATA

for i in range(9):
    for image, label_s in ds_test.take(1):  # capture the first batch in tf.dataset
      ax = plt.subplot(3, 3, i+1)
      image_i = np.uint8(255 * image[i])
      plt.imshow(image_i)
      plt.title(labels[label_s.numpy()[i]])
      plt.axis('off')

## 3 Load model

In [ ]:
model = tf.keras.models.load_model(model_path)
model.summary()

## 4 Evaluation on test sets

### 4.1 Statistics output

In [ ]:
# statistics output when evaluated on the test data
 
loss, acc = model.evaluate(ds_test)
print("Model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
em.cnn_statistics(model,ds_test)

### 4.2 Confusion matrix 

In [ ]:
em.ConfusionMatrix('DenseNet_TL_all_lr_0.0001_1.hdf5', model, ds_test, num_classes)


### 4.3 Show and save Misclassified examples

In [ ]:
y_true = np.concatenate([y for x, y in ds_test], axis=0)
y_pred = model.predict(ds_test).argmax(axis=1)
x_test = np.concatenate([x for x, y in ds_test], axis=0)
# x_test = x_test/255.0
misclassified_idx = np.where(y_pred != y_true)[0]
print("Number of misclassified images: ", misclassified_idx.size)


In [ ]:
# misclassified_idx

In [ ]:
# Save misclassified images in one folder


for i in misclassified_idx:
    fig = plt.figure(figsize=(2.24,2.24)) 
    plt.axis('off')
    plt.grid(False)
    x_test[i] = np.uint8(255 * x_test[i])
    plt.tight_layout(pad=0) # make the image full image (no padding)
    plt.imshow(x_test[i].astype(np.uint8), cmap='gray')
    # plt.title("%s pred as: %s" % (labels[y_true[i]], labels[y_pred[i]]))
    fig_name = labels[y_true[i]] + "_pred_as_" + labels[y_pred[i]] + str(i) + ".jpg"
    fig_path = os.path.join(save_misclassified_images_path,fig_name)
    fig.savefig(fig_path)
    plt.close() # close the figure, to prevent the figure shows up in Notebook

    

In [ ]:
# Show one random misclassified example
i = np.random.choice(misclassified_idx)
fig = plt.figure(figsize=(10,8))
plt.grid(False)
plt.imshow(x_test[i].astype(np.uint8), cmap='gray')
plt.title("True label: %s, Predicted: %s" % (labels[y_true[i]], labels[y_pred[i]]));

In [ ]:
# Show one specific misclassified example
fig = plt.figure(figsize=(10,8))
plt.grid(False)
plt.imshow(x_test[6].astype(np.uint8), cmap='gray')
plt.title("True label: %s, Predicted: %s" % (labels[y_true[6]], labels[y_pred[0]]));